In [1]:
import numpy as np
import pandas as pd
import json
import re

In [2]:
transcripts_df = pd.read_csv('annotated_transcirpts_117C_v2.csv')

In [3]:
CLEANR = re.compile('<.*?>')
CLEANR2 = re.compile('[.*?]')
CLEANR3 = re.compile("s+")

In [4]:
def clean_text(text):
    temp = re.sub(CLEANR, '', text)
    temp = re.sub(CLEANR2, '', temp)
    temp = temp.replace('\n', ' ')
    temp = re.sub(CLEANR3, ' ', temp).strip()
    temp = temp.lower()
    return temp

In [5]:
transcripts_df['transcript'] = transcripts_df['transcript'].apply(clean_text)

In [16]:
transcripts_df.head()

,Unnamed: 0,meta,transcript,date_x,bill_number,congress,chamber,rollnumber,date_y,session,...,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,vote_result,vote_desc,vote_question,dtl_desc
0,99,"{'body': 'H', 'depth': 2, 'download_timeout': ...","congre ional record, volume 168 i ue 12 (wedne...",2022-01-23,HR1,117,House,61,2021-03-03,1,...,210,0.213,-0.977,0.55,0.498,-8.63,Passed,For the People Act,On Passage,NaN
1,100,"{'body': 'S', 'depth': 2, 'download_timeout': ...","congre ional record, volume 168 i ue 8 (wedne ...",2022-01-23,HR1,117,House,61,2021-03-03,1,...,210,0.213,-0.977,0.55,0.498,-8.63,Passed,For the People Act,On Passage,NaN
2,101,"{'body': 'S', 'depth': 2, 'download_timeout': ...","congre ional record, volume 168 i ue 7 (tue da...",2022-01-23,HR1,117,House,61,2021-03-03,1,...,210,0.213,-0.977,0.55,0.498,-8.63,Passed,For the People Act,On Passage,NaN
3,102,"{'body': 'S', 'depth': 2, 'download_timeout': ...","congre ional record, volume 168 i ue 7 (tue da...",2022-01-23,HR1,117,House,61,2021-03-03,1,...,210,0.213,-0.977,0.55,0.498,-8.63,Passed,For the People Act,On Passage,NaN
4,103,"{'body': 'H', 'depth': 2, 'download_timeout': ...","congre ional record, volume 168 i ue 48 (thur ...",2022-03-24,HR1,117,House,61,2021-03-03,1,...,210,0.213,-0.977,0.55,0.498,-8.63,Passed,For the People Act,On Passage,NaN


In [7]:
# Load the glove mode
vec_txt_file_lines = open('glove.6B.100d.txt', 'r').readlines()

glove = {}

for line in vec_txt_file_lines:
    line_split = line.split(' ')
    key = line_split[0]
    vec = np.array([float(v) for v in line_split[1:]])
    glove[key] = vec

In [17]:
X_naive_mean = []
Y_naive_mean_yea = []
Y_naive_mean_passed = []

for i, row in transcripts_df.iterrows():
    tx_vecs = []
    for w in row['transcript'].split(' '):
        if w in glove:
            tx_vecs.append(glove[w])

    X_naive_mean.append(np.mean(tx_vecs, axis=0))
    Y_naive_mean_yea.append(row['yea_count'])
    Y_naive_mean_passed.append(True if row['vote_result'] == 'Passed' else False)

In [64]:
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_naive_mean, Y_naive_mean_yea, test_size=0.2, random_state=42)

model = LinearSVC().fit(X_train, y_train)

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred)**0.5)

/home/timevdo/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.48257839721254353
74.60622527761484


In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X_naive_mean, Y_naive_mean_passed, test_size=0.2, random_state=42)

model = LinearSVC().fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

/home/timevdo/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.9982578397212544

In [32]:
set(y_pred)

{True}

In [59]:
from xgboost import XGBClassifier, XGBRegressor

X_train, X_test, y_train, y_test = train_test_split(
    X_naive_mean, Y_naive_mean_yea, test_size=0.2, random_state=42)

bst = XGBRegressor(n_estimators=200, learning_rate=0.1)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
print(mean_squared_error(y_test, y_pred)**0.5)

54.56465036485781
